# **NPFL087 Project**
## **Spanish <-> Greek speech dataset**

> **Objective**: achievement of a big enough dataset of parallel speech adherences in the order they were spoken from parallel speech data, via usage of recognizers of Spanish and Greek and alignments done at the transcription text level.

> **Author**: Adriana R.Flórez ([adrirflorez@gmail.com](mailto:adrirflorez@gmail.com))

> **Course**: _NPFL087_ Statistical Machine Translation



In [ ]:
# 🧠 Install WhisperX and related audio packages
!pip install pydub
!pip install git+https://github.com/m-bain/whisperx.git
!apt-get install -y ffmpeg

# 🧠 Install Faster-Whisper
!pip install git+https://github.com/SYSTRAN/faster-whisper

# 🧰 Install setuptools-rust (required for some Whisper/WhisperX deps)
!pip install setuptools-rust

# 🔐 Hugging Face utility
!pip install hf_xet

# 📖 HunAlign
!apt-get install -y build-essential
!wget ftp://ftp.mokk.bme.hu/Hunglish/src/hunalign/latest/hunalign-1.1.tgz
!tar zxvf hunalign-1.1.tgz
%cd hunalign-1.1/src/hunalign
!make
%cd /content

  Cloning https://github.com/m-bain/whisperx.git to /tmp/pip-req-build-6j9sowxo
  Running command git clone --filter=blob:none --quiet https://github.com/m-bain/whisperx.git /tmp/pip-req-build-6j9sowxo
  Resolved https://github.com/m-bain/whisperx.git to commit 5012650d0f3d8966f3ea517762f952a624996d32
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.4/37.4 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 103.4 MB/s eta 0:00:00
   ━━

In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import torch
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
import numpy
import pyannote.audio
import faster_whisper
import whisperx

print("Dependency set-up checks:")
print("✅ PyTorch:", torch.__version__)
print("✅ CUDA available: ", torch.cuda.is_available())

Dependency set-up checks:
✅ PyTorch: 2.6.0+cu124
✅ CUDA available:  True


# **1. Data**

Input data should be a collection of .wav audio files found in a ./data/ directory, in a specific subdirectory identified with the audio's ISO-2 language code.

Output data is stored in .json, .srt and .txt formats, found in a ./transcriptions/ directory, in a specific subdirectory identified with the same language's code. Each transcription file has the same name as the .wav file it transcribes.

Parallel Greek <--> Spanish speech data has been downloaded from the VoxPopuli dataset for both languages, and preprocessed in order to achieve significant segments in Spanish and find their respective parallel segments in Greek, as well as a manifest "parallel_el-es.tsv" file that shows the correspondence between the two language audio files:



```
el	es
data\el_es\el\20090401-0900-PLENARY-11-el_es_20090401-15-32-14_5.wav	data\el_es\es\20090401-0900-PLENARY-11-el_es_20090401-15-32-14_5.wav

data\el_es\el\20090401-0900-PLENARY-11-el_es_20090401-16-03-40_1.wav	data\el_es\es\20090401-0900-PLENARY-11-el_es_20090401-16-03-40_1.wav

data\el_es\el\20090401-0900-PLENARY-11-el_es_20090401-16-03-40_4.wav	data\el_es\es\20090401-0900-PLENARY-11-el_es_20090401-16-03-40_4.wav
```



## 1.1 **Data validation**

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import shutil
import zipfile
import os

def clean(path):
    if not os.path.exists(path):
        print(f"Directory '{path}' does not exist.")
        return

    for filename in os.listdir(path):
        file_path = os.path.join(path, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)  # remove file or symbolic link
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)  # remove directory and its contents
            print(f"> Deleted: {file_path}")
        except Exception as e:
            print(f"[!]Failed to delete {file_path}. Reason: {e}")

In [ ]:
extract_path = '/content/data'
results_path = '/content/transcriptions'
#clean(extract_path)
#clean(results_path)

el_data = '/content/drive/MyDrive/Colab Notebooks/el.zip'
es_data = '/content/drive/MyDrive/Colab Notebooks/es.zip'
el_transcripts = '/content/drive/MyDrive/Colab Notebooks/transcriptions/el.zip'
es_transcripts = '/content/drive/MyDrive/Colab Notebooks/transcriptions/es.zip'

os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(el_data, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

with zipfile.ZipFile(es_data, 'r') as zip_ref:
  zip_ref.extractall(extract_path)

with zipfile.ZipFile(el_transcripts, 'r') as zip_ref:
  zip_ref.extractall(results_path + "/el")

with zipfile.ZipFile(es_transcripts, 'r') as zip_ref:
  zip_ref.extractall(results_path + "/es")

In [ ]:
def load(lang):
    """Loads input audio data paths from data directory and sets up the
    corresponding output data directories for the respective audio
    language code(s)."""

    dirs = {"data": "./data", "output": "./transcriptions"}
    os.makedirs(dirs["output"], exist_ok=True)

    files = {lang: []}

    out_lang_dir = os.path.join(dirs["output"], lang)
    os.makedirs(out_lang_dir, exist_ok=True)

    lang_dir = os.path.join(dirs["data"], lang)

    if os.path.exists(lang_dir):
      for f in os.listdir(lang_dir):
        if f.startswith('.') or f.startswith('__') or f == '.ipynb_checkpoints':
          continue

        file_path = os.path.join(lang_dir, f)
        output_path = os.path.join(out_lang_dir,
                                              f.replace(".wav", ".json"))

        # for each audio file, we save its (input) path,
        # corresponding output path, and language code
        file_info = {"input": file_path,
                              "output": output_path,
                              "lang": lang}
        files[lang].append(file_info)

    return files

In [ ]:
from google.colab import files
import shutil

def download(lang):
    folder_path = f"./{lang}"
    zip_path = f"./{lang}.zip"

    if not os.path.exists(folder_path):
        print(f"❌ Directory {folder_path} does not exist.")
        return

    shutil.make_archive(f"./{lang}", 'zip', folder_path)
    try:
        files.download(zip_path)
        print(f"⬇️ Download started for {zip_path}")
    except Exception as e:
        print(f"❌ Failed to download {zip_path}: {e}")
    print(f"✅ Downloaded transcriptions in [{lang}]!")

def export_json(result, input_file):
    path = input_file["output"]
    with open(path, "w", encoding="utf-8") as f:
        json.dump(result, f, ensure_ascii=False, indent=4)

def export_srt(result, input_file):
    def format_timestamp(seconds):
        hrs, rem = divmod(seconds, 3600)
        mins, secs = divmod(rem, 60)
        millis = (secs - int(secs)) * 1000
        return f"{int(hrs):02}:{int(mins):02}:{int(secs):02},{int(millis):03}"

    path = input_file["output"].replace("json", "srt");
    with open(path, "w", encoding="utf-8") as f:
        for i, segment in enumerate(result["segments"]):
            f.write(f"{i+1}\n")
            start = segment['start']
            end = segment['end']
            text = segment['text'].strip()
            f.write(f"{format_timestamp(start)} --> {format_timestamp(end)}\n{text}\n\n")

def export_audacity(f):
    srt_path = f["output"].replace("json", "srt")
    out_path = f["output"].replace(".json", "_labels.txt");

    with open(srt_path, "r", encoding="utf-8") as f:
        lines = f.read().splitlines()

    entries = []
    i = 0
    while i < len(lines):
        if lines[i].isdigit():
            start, end = lines[i + 1].split(" --> ")
            text = lines[i + 2]

            def to_sec(t):
              import re
              # HH:MM:SS,mmm with regex
              match = re.match(r"(\d+):(\d+):(\d+),(\d+)", t)
              if not match:
                  raise ValueError(f"Invalid timestamp format: {t}")
              h, m, s, ms = map(int, match.groups())
              return h * 3600 + m * 60 + s + ms / 1000.0

            entries.append(f"{to_sec(start)}\t{to_sec(end)}\t{text}")
            i += 4  # next entry
        else:
            i += 1

    with open(out_path, "w", encoding="utf-8") as f:
        f.write("\n".join(entries))

def export(result, f):
  export_json(result, f)
  export_srt(result, f)
  export_audacity(f)
  print(f"💾 Saved transcriptions > {f['output'].replace('json', '*')}\n")

def export_plain(f):
    txt_path = f["output"].replace("json", "txt")
    out_path = f["output"].replace(".json", "_plain.txt");

    with open(txt_path, 'r', encoding='utf-8') as inf, open(out_path, 'w', encoding='utf-8') as outf:
        for line in inf:
            parts = line.strip().split('\t')
            if len(parts) == 3:
                outf.write(parts[2] + '\n')

# **2. Whisper**

For a maximum of 2 languages, it combines usage of both WhisperX for alignment and FasterWhisper for transcription.

Transcription is done on a language-by-language basis, so only one model is loaded at once. This model can either be a 'large-v3', 'large-v2'... supported by Whisper itself, or a custom model from HuggingFace.

In [ ]:
import os
import json
import gc
from faster_whisper import WhisperModel
from transformers import WhisperForConditionalGeneration, WhisperProcessor

# global settings
audio = ".wav"
device = "cpu"
compute = "int8"

def setup(lang, model_name="large-v3", is_custom=False):
    """Loads and sets up FasterWhisper and WhisperX model for transcription and alignment
    of audio in a specific language."""

    print(f"\t💻 FasterWhisper model [{model_name}] for speech transcription in [{lang}]")

    if is_custom:
      custom_model = WhisperForConditionalGeneration.from_pretrained(model_name, device_map=device)
      custom_processor = WhisperProcessor.from_pretrained(model_name)
      model = (custom_model, custom_processor)
    else:
      model = WhisperModel(model_name, device=device, compute_type=compute)

    almodel, almetadata = whisperx.load_align_model(lang, device=device)
    model = {
        "device": device,
        "audio": audio,
        "aligner": (almodel.half(), almetadata),
        "model": model,
    }

    return model

def setdown(whisper_model, lang):
    print(f"💻 Setting down: Cleaning Whisper models and cache for [{lang}]...\n\n")
    del whisper_model["model"]
    del whisper_model["aligner"]

    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

# **3. Transcribing audio**

Some functions for audio processing and normalization:

In [ ]:
import numpy as np
import torchaudio
import tempfile
from faster_whisper.vad import get_speech_timestamps, collect_chunks, VadOptions
from pydub import AudioSegment
from pathlib import Path

def is_valid_wav(path):
    try:
        with open(path, 'rb') as f:
            return f.read(4) == b'RIFF'
    except:
        return False

def process_audio(path, target_sr = 16000, target_db = -20.0):
    """Loads and normalizes audio, returns waveform, duration, and saved path."""
    audio = AudioSegment.from_file(path)

    change_in_dBFS = target_db - audio.dBFS
    normalized = audio.apply_gain(change_in_dBFS)

    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".wav")
    normalized.export(temp_file.name, format="wav")
    temp_path = temp_file.name

    waveform, sr = torchaudio.load(temp_path)
    if sr != target_sr:
        waveform = torchaudio.transforms.Resample(sr, target_sr)(waveform)
    duration = len(audio) / 1000.0  # in seconds

    return temp_path, duration, waveform

def split_audio_file(path: str, chunk_ms: int = 10000, overlap_ms: int = 1000):
    """Splits audio into overlapping chunks (in ms). Returns [(path, offset)]."""
    audio = AudioSegment.from_file(path)
    chunks = []

    for i in range(0, len(audio), chunk_ms - overlap_ms):
        chunk = audio[i:i + chunk_ms]
        offset_sec = i / 1000.0

        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".wav")
        chunk.export(temp_file.name, format="wav")
        chunks.append((temp_file.name, offset_sec))

    return chunks


Transcription is done either in chunks (parallelized) or directly, either for a pretrained HuggingFace model or a Whisper-based one:

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.notebook import tqdm

def get_segments(model, audio_path, lang, is_custom, duration, th_s=30, chunk_size=10, batch_size=8, device="cuda"):
    # custom model
    if is_custom:
        custom_model, custom_processor = model
        if device == "cuda":
            custom_model = custom_model.half()
        custom_model = custom_model.to(device)
        return get_segments_custom(custom_model, custom_processor, audio_path, lang)

    # too big audio
    elif duration > th_s:
      return get_segments_in_chunks(model, audio_path, lang)

    # plain fasterwhisper transcription
    with torch.no_grad():
      segments, _ = model.transcribe(
          audio_path,
          language=lang,
          beam_size=5,
          #chunk_size=chunk_size,
          #batch_size=batch_size
      )

    result = []
    for seg in segments:
        result.append({
            "start": seg.start,
            "end": seg.end,
            "text": seg.text.strip()
        })
    return result

def get_segments_custom(hf_model, processor, audio_path, lang, chunk_length_s=10):

    speech_array, sampling_rate = torchaudio.load(audio_path)
    resampler = torchaudio.transforms.Resample(sampling_rate, 16000)
    speech_array = resampler(speech_array).squeeze().numpy()

    chunk_size = int(chunk_length_s * 16000)
    num_chunks = (len(speech_array) + chunk_size - 1) // chunk_size

    segments = []

    for i in range(num_chunks):
        start = i * chunk_size
        end = min(start + chunk_size, len(speech_array))
        chunk = speech_array[start:end]

        inputs = processor(chunk, sampling_rate=16000, return_tensors="pt", padding=True)
        input_features = inputs.input_features
        attention_mask = inputs.get("attention_mask", None)

        if hf_model.dtype == torch.float16:
            input_features = input_features.half()
        input_features = input_features.to(hf_model.device)

        if attention_mask is not None:
            attention_mask = attention_mask.to(hf_model.device)

        with torch.no_grad():
            predicted_ids = hf_model.generate(
                input_features,
                attention_mask=attention_mask,
                max_length=448,  # optional: control decoding length
            )

        transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

        segment = {
            "start": start / 16000,  # start time in seconds
            "end": end / 16000,      # end time in seconds
            "text": transcription.strip()
        }

        segments.append(segment)

    return segments

def get_segments_in_chunks(model, path, lang, max_chunks=None, workers=4):
    chunks = split_audio_file(path)
    results = []
    for idx, (chunk_path, offset) in tqdm(enumerate(chunks), total=min(len(chunks), max_chunks or len(chunks)), desc="chunk"):
        if max_chunks and idx >= max_chunks:
            break

        with torch.no_grad():
            segs, _ = model.transcribe(chunk_path, language=lang)

        for seg in segs:
            results.append({
                "start": seg.start + offset,
                "end": seg.end + offset,
                "text": seg.text.strip()
            })

        os.remove(chunk_path)

    return results

# **4. Transcribe**

For each language, we load the respective model and input files, retrieve their transcriptions after processing the audio, and align the timestamped segments with the audios. These results are saved onto the respective output files.

In [ ]:
def whisper_transcribe(model, f, custom=False):
    path, duration, audio = process_audio(f["input"])
    segments = get_segments(model["model"], path, f["lang"], custom, duration)
    return segments, audio

def whisper_align(model, segments, f, waveform):
    """Carries out alignment task with a given Whisper model, on an audio file."""

    align_model, metadata = model["aligner"]
    device = model["device"]

    if waveform.ndim == 1:
        waveform = waveform.unsqueeze(0)

    waveform = waveform.to(device=device, dtype=next(align_model.parameters()).dtype)

    # alignment is done with whisperx
    aligned_segments = whisperx.align(
        segments,
        align_model,
        metadata,
        waveform,
        device=device,
        return_char_alignments=False  # if not needed
    )

    #print(f"📜 Transcribed and aligned {f['input']}, segments: {len(segments)}")
    return aligned_segments

In [ ]:
import whisperx

langs_models = {
    #"el": "Sandiago21/whisper-large-v2-greek",
    "es": "Systran/faster-whisper-large-v3"
}

def transcribe_all(langs_models=langs_models):
  for lang, name in langs_models.items():
      # 1. load files for the language and check for their validity
      files = load(lang)
      invalid_files = [p for p in Path(f"./data/{lang}").glob("*.wav") if not is_valid_wav(p)]
      files = get_files_to_transcribe(files, lang)

      # 2. load transcription and alignment model for the language
      is_custom = (lang == "el")
      whisper_model = setup(lang, model_name=name, is_custom=is_custom)
      print(f"\n\t > 📛(!) Found {len(invalid_files)} invalid WAV files in [{lang}].")

      # 3. transcribe + align for all files
      for f in tqdm(files[lang], desc=f"[{lang}]"):

        if Path(f['input']) not in invalid_files:
            text, audio = whisper_transcribe(whisper_model, f, custom=is_custom)
            a = whisper_align(whisper_model, text, f, audio)

            # 4. save results for a file
            export(a, f)
        else:
          print(f"> ⚠️(!) Skipping transcription of invalid .wav file > {f['input']}")

      # 5. free memory
      setdown(whisper_model, lang)

      # 6. download results
      download(lang)

transcribe_all()

	💻 FasterWhisper model [Systran/faster-whisper-large-v3] for speech transcription in [es]

	 > 📛(!) Found 0 invalid WAV files in [es].


[es]:   0%|          | 0/33 [00:00<?, ?it/s]

💾 Saved transcriptions > ./transcriptions/es/20090916-0900-PLENARY-3-es_20090916-09-15-01_1.*

💾 Saved transcriptions > ./transcriptions/es/20090914-0900-PLENARY-14-es_20090914-19-31-32_9.*

💾 Saved transcriptions > ./transcriptions/es/20091215-0900-PLENARY-16-es_20091215-22-42-51_4.*

💾 Saved transcriptions > ./transcriptions/es/20090915-0900-PLENARY-10-es_20090915-17-38-36_7.*

💾 Saved transcriptions > ./transcriptions/es/20091125-0900-PLENARY-18-es_20091125-21-05-39_6.*

💾 Saved transcriptions > ./transcriptions/es/20090506-0900-PLENARY-12-es_20090506-16-39-17_6.*

💾 Saved transcriptions > ./transcriptions/es/20090401-0900-PLENARY-13-es_20090401-18-50-27_3.*

💾 Saved transcriptions > ./transcriptions/es/20090916-0900-PLENARY-3-es_20090916-10-09-51_9.*

💾 Saved transcriptions > ./transcriptions/es/20090715-0900-PLENARY-12-es_20090715-17-13-08_9.*

💾 Saved transcriptions > ./transcriptions/es/20090421-0900-PLENARY-17-es_20090421-21-56-13_14.*

💾 Saved transcriptions > ./transcriptions

chunk:   0%|          | 0/4 [00:00<?, ?it/s]

💾 Saved transcriptions > ./transcriptions/es/20090421-0900-PLENARY-17-es_20090421-22-18-31_0.*

💾 Saved transcriptions > ./transcriptions/es/20091215-0900-PLENARY-16-es_20091215-22-40-33_2.*

💾 Saved transcriptions > ./transcriptions/es/20090424-0900-PLENARY-4-es_20090424-09-40-44_3.*

💾 Saved transcriptions > ./transcriptions/es/20091215-0900-PLENARY-3-es_20091215-10-22-28_0.*

💾 Saved transcriptions > ./transcriptions/es/20091111-0900-PLENARY-11-es_20091111-17-28-50_0.*

💾 Saved transcriptions > ./transcriptions/es/20090504-0900-PLENARY-10-es_20090504-19-11-16_3.*

💾 Saved transcriptions > ./transcriptions/es/20091217-0900-PLENARY-7-es_20091217-11-42-08_0.*

💾 Saved transcriptions > ./transcriptions/es/20091020-0900-PLENARY-13-es_20091020-17-39-09_3.*

💾 Saved transcriptions > ./transcriptions/es/20091020-0900-PLENARY-4-es_20091020-10-39-16_2.*

💾 Saved transcriptions > ./transcriptions/es/20091215-0900-PLENARY-16-es_20091215-22-42-51_8.*

💾 Saved transcriptions > ./transcriptions/es

In [ ]:
def get_files_to_transcribe(audio_files, lang, transcripts_dir="./transc_el/"):
    """Filters the audio files to only include those for which corresponding transcripts already exist."""
    filtered_files = {f"{lang}" : []}

    for file_info in audio_files.get("es", []):
        es_path = file_info["input"]
        filename = os.path.basename(es_path)

        # Map the Spanish filename to the expected Greek transcription filename
        base = filename.replace("-es_", "-el_es_").replace(".wav", "")
        json = os.path.join(transcripts_dir, base + ".json")
        srt = os.path.join(transcripts_dir, base + ".srt")

        if os.path.exists(json) and os.path.exists(srt):
            filtered_files[lang].append(file_info)

    return filtered_files

## **5. Clean**

Once transcriptions of all parallel audio files have been completed, clean them to remove any potential hallucinations or gibberish coming from faulty Whisper transcription.

In [ ]:
!pip install pysrt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.4/104.4 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pysrt: filename=pysrt-1.1.2-py3-none-any.whl size=13443 sha256=61aa053c6d1075e864dedd4dd4473c7fb953cad607636908fc2129d04e6e2693
  Stored in directory: /root/.cache/pip/wheels/2d/b2/df/ea10959920533975b4a74a25a35e6d79655b63f3006611a99f
Successfully built pysrt


In [ ]:
import re
import pysrt
import os

# audio files that have been transcribed
es_files = load("es")
el_files = load("el")

audios = [es_files, el_files]
langs = ["es", "el"]

def get_clean_subdir(path):
    dir_path, filename = os.path.split(path)
    head, lang = os.path.split(dir_path)
    new_dir = os.path.join(head, 'cleaned', lang)
    os.makedirs(new_dir, exist_ok=True)
    return os.path.join(new_dir, filename)


def get_clean_transcriptions(audio_files, langs, transcripts_dir="./transcriptions/", cleaned_dir="cleaned"):
    """
    Retrieves .srt transcription files that match the given audio files for a specific language.
    """
    transc = {langs[0]: [], langs[1]: []}
    os.makedirs(transcripts_dir + cleaned_dir, exist_ok=True)

    for i, lang in enumerate(langs):
      for file_info in audio_files[i].get(lang, []):
          audio_path = file_info["input"]
          base = file_info["output"].replace(".json","")
          filename = os.path.basename(audio_path)

          srt_path = file_info["output"].replace(".json",".srt")
          txt_path = file_info["output"].replace(".json",".txt")

          output_srt_path = get_clean_subdir(srt_path)
          output_txt_path = get_clean_subdir(txt_path)

          if os.path.exists(srt_path):
                # clean .srt file
                subs = pysrt.open(srt_path, encoding='utf-8')
                sentences = []
                for sub in subs:
                    cleaned = clean(sub.text)
                    if cleaned and len(cleaned.split()) > 2:
                        sentences.append(cleaned)

                # save .srt file
                with open(output_txt_path, 'w', encoding='utf-8') as out:
                    out.write('\n'.join(sentences))

                # save file info
                cleaned_file = {"srt": output_srt_path,
                                "txt": output_txt_path,
                                "sentences": sentences}
                transc[lang].append(cleaned_file)

    return transc

def clean(text):
    text = text.replace('\n', ' ').strip()
    text = text.replace('\n', ' ').strip()
    text = re.sub(r'(.)\1{3,}', '', text)  # long repeated chars
    text = re.sub(r'\b(\w+)(\s+\1\b)+', r'\1', text, flags=re.IGNORECASE)  # repeated words
    text = re.sub(r'[^\w\s.,;!?¡¿-]', '', text, flags=re.UNICODE)  # allow Unicode words
    return text.strip()

transcriptions = get_clean_transcriptions(audios, langs)


In [ ]:
shutil.make_archive('cleaned_transcriptions_es', 'zip', '/content/transcriptions/cleaned/es')
files.download('cleaned_transcriptions_es.zip')

shutil.make_archive('cleaned_transcriptions_el', 'zip', '/content/transcriptions/cleaned/el')
files.download('cleaned_transcriptions_el.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **5. Align**

Once transcriptions of all parallel audio files have been completed, carry out their respective alignment with HunAlign:

In [ ]:
import pandas as pd
import subprocess

def get_id(filepath):
    filename = os.path.basename(filepath)
    match = re.search(r'(\d{8}-\d{2}-\d{2}-\d{2}_\d+)', filename)
    return match.group(1) if match else None

def get_parallel_files(greek_files, spanish_files):
    """Match Greek and Spanish files based on shared identifier."""
    greek_map = {get_id(g['srt']): g for g in greek_files}
    spanish_map = {get_id(s['srt']): s for s in spanish_files}

    matches = []
    for identifier, greek_file in greek_map.items():
        spanish_file = spanish_map.get(identifier)
        if spanish_file:
            matches.append((greek_file, spanish_file))

    return matches


def hun_align(transc, dictionary_dir="/dev/null", output_dir="./aligned/", min_score=0.85, hun_exe="hunalign-1.1/src/hunalign/hunalign"):
    """Aligns Greek-Spanish sentence pairs from cleaned transcriptions using HunAlign.
    """
    os.makedirs(output_dir, exist_ok=True)
    aligned_results = []
    al_path = os.path.join(output_dir, "alignments.txt")

    parallel = get_parallel_files(transc["el"], transc["es"])

    for el_info, es_info in parallel:
        base_name = os.path.splitext(os.path.basename(el_info["txt"]))[0]
        out_path = os.path.join(output_dir, base_name + ".aligned.txt")

        command = [
            hun_exe, dictionary_dir, "-text",
            es_info["txt"], el_info["txt"]
        ]

        with open(out_path, "w", encoding="utf-8") as out_file:
            result = subprocess.run(command, stdout=out_file, stderr=subprocess.PIPE)

        # parse output
        with open(out_path, "r", encoding="utf-8") as aligned_file:
            for line in aligned_file:
                if "|||" not in line:
                    continue
                parts = line.strip().split("|||")
                if len(parts) != 3:
                    continue
                try:
                    score = float(parts[0].strip())
                    if score < min_score:
                        continue
                    es_sent = parts[1].strip()
                    el_sent = parts[2].strip()
                    aligned_results.append((base_name, score, es_sent, el_sent))

                except:
                    continue

    with open(al_path, "w", encoding="utf-8") as out:
      out.write("file\tscore\tes\tsel\n")
      for fname, score, es, el in aligned_results:
        out.write(f"{fname}\t{score:.2f}\t{es}\t{el}\n")

    return aligned_results

hun_align(transcriptions)
shutil.make_archive('alignments', 'zip', '/content/aligned')
files.download('alignments.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>